In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
import numpy as np
import regex as re

In [11]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    sentences = [sentence.strip() for sentence in re.split(r'(?<=[.!?])\s+', text) if sentence.strip()]
    return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)


In [12]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [13]:
# Creating Input Sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        

In [15]:
# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
X,y = input_sequences[:, :-1], input_sequences[:,-1]

# Converting target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes = total_words)

In [16]:
# Defining NN model
model = Sequential()
model.add(Embedding(total_words, 10, input_length = max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
# Train the model
model.fit(X, y, epochs=150, verbose=1)


Epoch 1/150
51/51 [==============================] - 3s 19ms/step - loss: 6.2007 - accuracy: 0.0467
Epoch 2/150
51/51 [==============================] - 1s 19ms/step - loss: 5.7962 - accuracy: 0.0559
Epoch 3/150
51/51 [==============================] - 1s 19ms/step - loss: 5.7289 - accuracy: 0.0528
Epoch 4/150
51/51 [==============================] - 1s 19ms/step - loss: 5.6990 - accuracy: 0.0547
Epoch 5/150
51/51 [==============================] - 1s 19ms/step - loss: 5.6639 - accuracy: 0.0553
Epoch 6/150
51/51 [==============================] - 1s 19ms/step - loss: 5.6234 - accuracy: 0.0498
Epoch 7/150
51/51 [==============================] - 1s 19ms/step - loss: 5.5760 - accuracy: 0.0565
Epoch 8/150
51/51 [==============================] - 1s 19ms/step - loss: 5.5244 - accuracy: 0.0571
Epoch 9/150
51/51 [==============================] - 1s 19ms/step - loss: 5.4668 - accuracy: 0.0590
Epoch 10/150
51/51 [==============================] - 1s 19ms/step - loss: 5.4046 - accuracy: 0.0614

In [19]:
# Generating next word predictions
seed_text = "Pizza have different"
next_words = 5
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen = max_sequence_len - 1, padding = 'pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word
print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 27ms/step
Next predicted words: Pizza have different become a symbol of comfort
